<a href="https://colab.research.google.com/github/sallyelbaz/Automated-Multi-Source-Data-Pipeline-/blob/main/Multi_Source_Data_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


##Automated Multi-Source Data Pipeline

Problem: Data is fragmented across many internal systems. Analysts manually pull data, causing
delays and errors.

Solution: Build an automated pipeline that connects to multiple data sources (APIs, SQL database, flat
files), cleans the data, and loads it into a cloud warehouse (BigQuery/Snowflake). Set up scheduled
automation.


Tech Stack: Python, SQL, Airflow, dbt, BigQuery/Snowflake, GitHub Actions
Expected Business Impact: Reduced manual reporting effort by 90%. Enabled real time
decision making.